## Representación de las tasas de paro en España 

Vamos a representar las distintas tasas de paro en España desagregadas por Comunidad Autónoma, edad y sexo correspondientes al segundo trimestre de 2025 (el más reciente). Para ello, hemos extraído lainformación de un conjunto de datos del INE (https://www.ine.es/). En segundo lugar, hemos utilizado la base de datos trabajada en clase *(SP_provincias.geojson)* para obtener las geometrías de cada una de las comunidades autónomas.

In [ ]:
# Importación de las librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import warnings 
import plotly.express as px
import plotly.graph_objects as go 
import plotly.io as pio
from datetime import date
from datetime import datetime
import plotnine
from plotnine import *
from dash import dash, dcc, html
import folium
import mapclassify

### 1. Tratamiento de los datos

Vamos a realizar la importación de los datos y la posible manipulación de estos para poder, posteriormente, dibujar los gráficos, hacer nuestro análisis visual de los datos y extraer conclusiones.

In [ ]:
# Importación del csv  tasa_paro
dataset = pd.read_csv('tasa_paro.csv', sep=';',encoding='latin-1')

In [ ]:
# Primer visatzo al dataset
dataset.head()

In [ ]:
# Renombramos titulo, pues es demasiado largo
dataset.rename(columns={'Comunidades y Ciudades Autónomas':'comunidad_autonoma'}, inplace=True)

In [ ]:
# Vemos el tipo de dato de cada uno de 
dataset.info()

In [ ]:
# Transformamos el total en número, para poder hacer groupby si lo consideramos necesario, para ello debemos substituir las , por . y los .. por NA (pues se consideran valores nulos)
dataset['tasa_paro'] = pd.to_numeric(dataset['Total'].astype(str).str.strip().str.replace(',','.',regex=False).replace('..', pd.NA))

In [ ]:
# Cambiamos a formato fecha la variable periodo ¡¡¡¡¡ SE PUEDE LIMPIAR !!!!!!
tmp = dataset['Periodo'].str.extract(r'(?P<year>\d{4})T(?P<q>[1-4])').astype(int) # capturamos los 4 priemros digisot que coresponden al año y la letra T y un valor del 1-4 que corresponden a los trimestres
month = (tmp['q'] - 1) * 3 + 1  # Q1->1, Q2->4, Q3->7, Q4->10 /convertimos el trimestre a mes de inicio del trimestre
dataset['Periodo_dt'] = pd.to_datetime(dict(year=tmp['year'], month=month, day=1)) # Crea una fecha real con el primer dia de cada trimestre
dataset['Periodo_dt'] = dataset['Periodo_dt'].dt.to_period('Q').astype(str) # lo pasamos a trimestre pero formato date

In [ ]:
dataset.info()

In [ ]:
dataset.head()

In [ ]:
# Ahora debemos eliminar las variables total y periodo, pues no las necesitamos para nada
dataset.drop(['Periodo','Total'],axis=1,inplace=True)

Ahora, puesto que necesitamos la *geometry* (coordenadas) de cada comunidad autónoma para poder dibujar un mapa de la peninsula ibérica, importamos los datos usados en clase *SP_provincias.geojson* para tratar de hacer un merge con el conjunto de datos principal. 


In [ ]:
# Importación del archivo geojson
geo_prv= r'SP_provincias.geojson'
gdf = gpd.read_file(geo_prv)

In [ ]:
gdf.head()

In [ ]:
# Dropeamos las variabels CCC_2 y NAT2018, pues no las necesitamos. 
gdf.drop(['CC_2','NAT2018'],axis=1,inplace=True)


In [ ]:
# con la función dissolve(), para cada comunidad, tomamos todos los polígonos provinciales y los fusionamos en una única geometría  
gdf_ccaa = gdf.dissolve(by='NAME_1',  as_index=False)

# seguidamente podemos dropear la columna name_2
gdf_ccaa.drop(['NAME_2'],axis=1,inplace=True)

In [ ]:
gdf_ccaa

Ahora, lo que debemos hacer es unir ambos dataframes por la columna “comunidad autónoma”. Para ello, debemos asegurarnos de que ambas columnas tienen el mismo nombre y formato (acentos, mayúsculas/minúsculas, guiones, etc.).

In [ ]:
 # Debemos modificar los titulso de las comunidades autonomas
dataset['comunidad_autonoma'].value_counts()

In [ ]:
# Eliminamos los códigos de cada comunidad autonoma 
dataset['ccaa'] = (dataset['comunidad_autonoma']
                   .astype(str)
                   .str.replace(r'^\d{1,2}\s+', '', regex=True)
                   .str.strip())

In [ ]:
# Creamos un diccionario con los reemplazos para adaptarlo correctamente a la tabla gdf_ccaa
reemplazos = {
    'Rioja, La': 'La Rioja',
    'Navarra, Comunidad Foral de': 'Comunidad Foral de Navarra',
    'Madrid, Comunidad de': 'Comunidad de Madrid',
    'Murcia, Región de': 'Región de Murcia',
    'Balears, Illes': 'Islas Baleares',
    'Illes Balears': 'Islas Baleares',
    'Valenciana, Comunidad': 'Comunidad Valenciana',
    'Comunitat Valenciana': 'Comunidad Valenciana',
    'Catalunya': 'Cataluña',
    'Asturias, Principado de': 'Principado de Asturias',
    'Canarias' : 'Islas Canarias',
    'Castilla - La Mancha' : 'Castilla-La Mancha'
    
}
dataset['ccaa'] = dataset['ccaa'].replace(reemplazos) # reemplazamos en la tabal dataset
dataset = dataset[dataset['ccaa'] != 'Total Nacional']  # Eliminamos total Nacional, pues no lo usaremos

In [ ]:
dataset.drop(['comunidad_autonoma'],axis=1,inplace=True)

In [ ]:
gdf_ccaa.rename(columns = {'NAME_1' : 'ccaa'},inplace=True)

In [ ]:
# Hacemos el merge entre los dos datasets
data = dataset.merge(gdf_ccaa,
                     how='left',
                     left_on = 'ccaa',
                     right_on  = 'ccaa',
                     suffixes= ['_x','_y'])

In [ ]:
# Puesto que no disponemos ni de ceuta ni de melilla, vamos a eliminarlos
data = data[data['ccaa'] != 'Melilla']
data = data[data['ccaa'] != 'Ceuta']
data

### Gráfico 1: Paro por sexo en un mapa de España segundo trimestre 2025 

Vamos a dibujar un mapa interactivo de España que muestre, mediante una escala de color (coropleta), las tasas de paro por comunidad autónoma, con un desplegable para filtrar por sexo. A partir de ahí, podremos analizar el paro por sexo y por comunidad autónoma y comprobar si existen brechas entre comunidades y entre sexos.

In [ ]:
# Primero debemos filtrar para el segundo trimestre de 2025

data_2025_Q2_sexo = data[(data['Periodo_dt'] == '2025Q2') & (data['Edad'] == 'Total')]


In [ ]:
data_2025_Q2_sexo.head()

In [ ]:
# Para ello es necesario pasarlo a geoDataFarme
data_geo_sexo = gpd.GeoDataFrame(data_2025_Q2_sexo,geometry=data_2025_Q2_sexo['geometry'])

In [ ]:
# Escala común de colores para que no cambie al cambiar de sexo
vmin, vmax = data_geo_sexo['tasa_paro'].min(), data_geo_sexo['tasa_paro'].max()

# 1) Capa base: “Ambos sexos” (crea el mapa)
m = data_geo_sexo[data_geo_sexo['Sexo'] == 'Ambos sexos'].explore(
    column='tasa_paro',
    scheme='EqualInterval',
    k=7,
    legend=True,
    legend_kwds=dict(colorbar=True, caption='Tasa de paro (%)'),
    name='Ambos sexos',
    tooltip=['ccaa','tasa_paro'],
    cmap='Oranges',
    vmin=vmin, vmax=vmax,
    tiles='CartoDB positron',
    zoom_start=5,          # tamaño de vista (no es el tamaño del lienzo)
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# 2) Añade capa “Hombres”
data_geo_sexo[data_geo_sexo['Sexo'] == 'Hombres'].explore(
    m=m,
    column='tasa_paro',
    scheme='EqualInterval', k=7,
    legend=False,
    name='Hombres',
    tooltip=['ccaa','tasa_paro'],
    cmap='Oranges',
    vmin=vmin, vmax=vmax,
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# 3) Añade capa “Mujeres”
data_geo_sexo[data_geo_sexo['Sexo'] == 'Mujeres'].explore(
    m=m,
    column='tasa_paro',
    scheme='EqualInterval', k=7,
    legend=False,
    name='Mujeres',
    tooltip=['ccaa','tasa_paro'],
    cmap='Oranges',
    vmin=vmin, vmax=vmax,
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# Control de capas (el "filtro")
folium.LayerControl(collapsed=False).add_to(m)

m

#### Conclusiones

Para analizar el mapa de España según la tasa de paro por comunidad autónoma, comenzaremos examinando el conjunto de ambos sexos (total) y, posteriormente, realizaremos el análisis desagregado por sexo.

Al observar el total, se aprecia que las mayores tasas de desempleo se concentran en el sur, en las comunidades autónomas de Extremadura (15,48%) y Andalucía (14,85%), así como en Canarias (13,33%). Por el contrario, las comunidades con menores tasas se sitúan principalmente en el norte peninsular —Cantabria (7,13%), País Vasco (7,13%) y Aragón (7,64%), entre otras—, además de la Comunidad de Madrid (7,74%). Este patrón puede relacionarse con diferencias históricas en el tejido productivo y la estructura del mercado laboral: el norte ha estado tradicionalmente más conectado e industrializado, mientras que el sur ha presentado mayores dificultades en la creación de empleo.

Resulta especialmente relevante el comportamiento de la Comunidad de Madrid (7,74%): pese a su peso demográfico y económico y a su papel como capital, se sitúa entre las regiones con menor tasa de desempleo. Esto sugiere un mercado laboral más dinámico y diversificado (fuerte sector terciario y mayor capacidad de absorción de empleo cualificado).

Asimismo, conviene subrayar la presencia de Cantabria (7,13%) entre los valores más bajos del país. Su posición refuerza el patrón norteño de menores tasas y la distingue, a pesar de su menor tamaño, por una evolución del mercado laboral relativamente favorable frente a la media nacional

*Hombres*

Al analizar las tasas de desempleo masculinas aparece un patrón algo distinto al del total. Destaca el aumento relativo en las comunidades del norte —Galicia (8,65%), Asturias (8,30%) y País Vasco (8,26%)— frente al resto. Aun así, Extremadura (12,34%) y Andalucía (12,85%) continúan entre las tasas más elevadas.

*Mujeres*

Resalta la baja tasa femenina en el País Vasco (5,94%) y el ligero aumento en la Comunidad de Madrid (8,19%). En el extremo alto vuelven a aparecer Andalucía (17,17%) y Extremadura (19,01%)


*Conclusiones finales*

En prácticamente todas las comunidades autónomas, la tasa de paro femenina supera a la masculina, con diferencias más acusadas en el sur. Este patrón puede explicarse por la segmentación ocupacional (mayor peso femenino en actividades con más temporalidad y rotación) y por barreras de conciliación que dificultan la continuidad laboral. En contraste, el País Vasco presenta una brecha inversa (paro femenino inferior al masculino), coherente con una mayor inserción femenina en sectores estables (sanidad, educación, servicios avanzados) y una mayor exposición masculina a ramas cíclicas como la industria y la construcción.

### Gráfico 2: Paro por edad en un mapa de España segundo trimestre 2025 

Vamos a dibujar un mapa interactivo de España que muestre, mediante una escala de color (coropleta), las tasas de paro por comunidad autónoma, con un desplegable para filtrar por franja etaria. A partir de ahí, podremos analizar el paro por franja etaria y comunidad autónoma y observar las brechas existentes entre estas variables.

In [ ]:
# Creamos la base de datos para mostrar el mapa del segundo trimestre de 2025 por edad, cogiendo un solo sexo y eliminando el total en edad
data_2025_Q2_edad = data[(data['Periodo_dt'] == '2025Q2') & (data['Sexo'] == 'Ambos sexos')]


In [ ]:
data_2025_Q2_edad.head()

In [ ]:
# Para ello es necesario pasarlo a geoDataFarme
data_geo_edad = gpd.GeoDataFrame(data_2025_Q2_edad,geometry=data_2025_Q2_edad['geometry'])

In [ ]:
data_geo_edad['Edad'].value_counts()

In [ ]:
# Escala común de colores para que no cambie al cambiar de sexo
vmin, vmax = data_geo_edad['tasa_paro'].min(), data_geo_edad['tasa_paro'].max()

# 1) Capa base: “Total” (crea el mapa)
m = data_geo_edad[data_geo_edad['Edad'] == 'Total'].explore(
    column='tasa_paro',
    scheme='EqualInterval',
    k=7,
    legend=True,
    legend_kwds=dict(colorbar=True, caption='Tasa de paro (%)'),
    name='Total',
    tooltip=['ccaa','tasa_paro'],
    cmap='Blues',
    vmin=vmin, vmax=vmax,
    tiles='CartoDB positron',
    zoom_start=5,          # tamaño de vista (no es el tamaño del lienzo)
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# 2) Añade capa 'De 16 a 19 años'
data_geo_edad[data_geo_edad['Edad'] == 'De 16 a 19 años'].explore(
    m=m,
    column='tasa_paro',
    scheme='EqualInterval', k=7,
    legend=False,
    name='De 16 a 19 años',
    tooltip=['ccaa','tasa_paro'],
    cmap='Blues',
    vmin=vmin, vmax=vmax,
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# 3) Añade capa 'De 20 a 24 años'
data_geo_edad[data_geo_edad['Edad'] == 'De 20 a 24 años'].explore(
    m=m,
    column='tasa_paro',
    scheme='EqualInterval', k=7,
    legend=False,
    name='De 20 a 24 años',
    tooltip=['ccaa','tasa_paro'],
    cmap='Blues',
    vmin=vmin, vmax=vmax,
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# 4) Añade capa 'De 25 a 54 años'
data_geo_edad[data_geo_edad['Edad'] == 'De 25 a 54 años'].explore(
    m=m,
    column='tasa_paro',
    scheme='EqualInterval', k=7,
    legend=False,
    name='De 25 a 54 años',
    tooltip=['ccaa','tasa_paro'],
    cmap='Blues',
    vmin=vmin, vmax=vmax,
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)

# 5) Añade capa 'De 55 y más años'
data_geo_edad[data_geo_edad['Edad'] == 'De 55 y más años'].explore(
    m=m,
    column='tasa_paro',
    scheme='EqualInterval', k=7,
    legend=False,
    name='De 55 y más años',
    tooltip=['ccaa','tasa_paro'],
    cmap='Blues',
    vmin=vmin, vmax=vmax,
    highlight=False,
    style_kwds=dict(weight=0.3, fillOpacity=0.7)
)


# Control de capas (el "filtro")
folium.LayerControl(collapsed=False).add_to(m)

m

#### Conclusiones

Puesto que en el anterior gráfico ya hemos analizado las tasas de paro por comunidad autonoma en su conjunto (ambos sexos), en estas conclusiones pasaremos a analizar las tasas de paro directemente por franja etaria.

*De 16 a 19 años*

Si hablamos de las personas de entre 16 y 19 años —es decir, los más jóvenes en edad de trabajar—, sorprende la concentración de tasas de paro elevadas en las comunidades autónomas del norte; destacan el Principado de Asturias (66,64 %), la Comunidad Foral de Navarra (59,84 %) y Galicia (47,95 %). En lo que respecta a las tasas más bajas, se sitúan la Comunidad de Madrid (25,95 %), Castilla-La Mancha (34,54 %) y Castilla y León (35,96 %). En general, esta franja etaria registra tasas de paro bastante elevadas, en torno al 45 %. Al contrario que en las demás franjas de edad, podría interpretarse de forma positiva una tasa elevada, pues implicaría que los jóvenes de estas edades están mayoritariamente en formación y, por tanto, se contará con una juventud más cualificada.

*De 20 a 24 años*

En lo que respecta a las personas de entre 20 y 24 años, el patrón se asemeja más al del conjunto del desempleo total. Las tasas elevadas del norte se reducen notablemente y se concentran en comunidades del sur, en particular Andalucía (30,31%), Extremadura (36,87%) y Canarias (31,14%). Las menores tasas se registran en Cataluña (13,97 %), la Comunidad Foral de Navarra (14,08%) y la Comunidad de Madrid (15,51%). En el norte, se mantiene una tasa de paro alta en el Principado de Asturias (22,43%), si bien también se observa un descenso muy significativo respecto al grupo más joven. En estas edades predominan las primeras inserciones laborales; de ahí que las tasas converjan hacia los niveles globales y sean más bajas en aquellas comunidades autónomas con mayor actividad económica. Aún así, este grupo se mantiene en tasas elevadas en comparación con el total.

*De 25 a 54 años*

En cuanto a la franja etaria del grueso de la población activa, la distribución de las tasas de desempleo por comunidades autónomas se aproxima a la del conjunto de España. Las tasas más altas se concentran en el sur, destacando, una vez más, Andalucía (13,27 %), Extremadura (13,67 %) y Canarias (11,58 %). Destaca también el aumento del peso de Castilla-La Mancha (11,48 %) en el total, un factor poco relevante hasta ahora. En lo que respecta a las tasas más bajas, se sitúan en el norte y centro peninsular, con la Comunidad de Madrid (6,37 %) y el País Vasco (6,04 %) como referencias. En conjunto, las tasas de esta franja etaria son mucho más bajas que en las dos primeras, pues la mayoría ya ha finalizado sus estudios y acumula experiencia laboral, lo que facilita encontrar trabajo.

*De 55 y más años*

En relación con la franja etaria de las personas de mayor edad, destaca Castilla-La Mancha (14,76 %) con la mayor tasa de desempleo. Le siguen Extremadura (14,24 %), Andalucía (14,02 %) y Canarias (12,59 %). Otro aspecto reseñable es la situación de la Comunidad Valenciana (11,91 %) entre las más elevadas; probablemente se deba a una mayor proporción de población de edad avanzada respecto a otras comunidades. Entre las tasas más bajas destacan Aragón (4,74 %), la Comunidad Foral de Navarra (5,66 %) y Cataluña (5,68 %).

*Conclusiones finales*

En conjunto, los resultados muestran un claro gradiente por edad: las tasas son muy elevadas en 16–19 años, descienden de forma apreciable en 20–24 con las primeras inserciones laborales, alcanzan mínimos en 25–54 y vuelven a repuntar en 55 y más. Geográficamente, el patrón también se desplaza con la edad: en la juventud temprana los máximos se concentran en el norte (Asturias, Navarra, Galicia), mientras que a partir de 20–24 la mayor incidencia se desplaza al sur (Andalucía, Extremadura, Canarias) y se consolida en la edad central. De manera consistente, la Comunidad de Madrid y el País Vasco se mantienen entre las tasas más bajas, en tanto que Castilla-La Mancha gana peso en 25–54 y lidera en 55 y más; asimismo, la Comunidad Valenciana registra valores elevados en el tramo de mayor edad. Este comportamiento es coherente con la transición educativa-laboral en los más jóvenes, con factores estructurales del tejido productivo en la población central y con necesidades de recualificación en los mayores.


### 3. Grafico 3: Treemap tasa paro por edad, para analizar paro juvenil segundo trimestre 2025 

In [ ]:
data_treemap = data[(data['Periodo_dt'] == '2025Q2') & (data['Sexo'] != 'Ambos sexos') ]


In [ ]:
data_treemap

In [ ]:
fig = px.treemap(data_treemap,
                 title = ('TASA DE PARO POR COMUNIDAD AUTONOMA'),
                 height = 550,
                 width = 950,
                 path =['ccaa','Edad','Sexo'],
                 values = 'tasa_paro',
                 color = 'tasa_paro',
                 labels = {'tasa_paro':'tasa_paro'})

fig.show() 

#### Conclusiones Finales

El treemap por CCAA, Edad y Sexo confirma que la edad es el factor que más condiciona el desempleo y, sobre ese patrón, se mantiene una brecha de género persistente. El tramo 16–19 años registra las tasas más altas en ambos sexos (en torno al 40%) y es el único en el que los hombres superan levemente a las mujeres. A partir de 20–24 años la tasa desciende pero continúa elevada y se abre con claridad la desigualdad por sexo, con mujeres por encima de hombres. En la franja 25–54 los valores convergen hacia el promedio nacional, aunque la brecha entre sexos se mantiene pero de forma más reducida. En 55 y más años se observa un ligero repunte de la tasa y vuelve a ampliarse la diferencia, de nuevo con mujeres por encima. 

En conjunto, el gráfico refuerza dos ideas: la crítica vulnerabilidad juvenil (16–24) y una brecha de género sostenida a partir de los 20 años, con picos al inicio y al final de la vida laboral.